# Задание 3. Применение RNN для прогнозирования отношения в рецензиях на фильмы.

Импортируем нужные модули и слои для нейросети.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional
from keras.utils import pad_sequences
from keras.layers import Embedding
import numpy as np
import pandas as pd
np_seed = 73
np.random.seed(np_seed)

Импортируем датасет из 25000 рецензий IMDb, загружаем тренировочную и тестовую выборку.

In [ ]:
from keras.datasets import imdb
max_features = 5000 #Максимальное количество различных слов в импортируемых выборках.
(X_train, y_train), (X_test, y_test) = imdb.load_data(seed = np_seed, num_words = 5000)
print("Размерность тренировочной выборки:")
print(X_train.shape, y_train.shape)
print("Размерность тестовой выборки:")
print(X_test.shape, y_test.shape)

Размерность тренировочной выборки:
(25000,) (25000,)
Размерность тестовой выборки:
(25000,) (25000,)


Строим модель Sequential следующей структуры:
- Слой Embedding. Служит для понижения размерности входных данных.
- Слой LSTM внутри оболочки Bidirectional. LSTM = Long Short-Term Memory. Часть Keras RNN API. Слой, способный различать долгосрочные зависимости и работать с ними.
- Слой Dense с функцией активации "sigmoid".

In [ ]:
max_reviewlength = 500 #Максимальное количество слов в одной рецензии - остальное обрезается. При недостатке - восполняется пустыми словами.
LSTM_units = 64 #Выходная размерность слоя Embedding, входная размерность слоя LSTM.
X_train_clamped = pad_sequences(X_train, maxlen = max_reviewlength)
X_test_clamped = pad_sequences(X_test, maxlen = max_reviewlength)
def CreateModel():
  model = Sequential()
  model.add(Embedding(max_features, LSTM_units, input_length = max_reviewlength))
  model.add(Bidirectional(LSTM(LSTM_units)))
  model.add(Dense(1, activation = "sigmoid"))
  model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
  return model

Для обучения выбираем следующие параметры:
- batch_size - 64. Количество сэмплов для градиентного спуска.
- epochs - 3. Количество эпох модели.

In [ ]:
batch_size = 64
epochs = 3
model = CreateModel()
model.fit(X_train_clamped, y_train, batch_size = batch_size, epochs = epochs)

Epoch 1/3
391/391 [==============================] - 361s 899ms/step - loss: 0.4250 - accuracy: 0.7996
Epoch 2/3
391/391 [==============================] - 348s 890ms/step - loss: 0.2792 - accuracy: 0.8895
Epoch 3/3
391/391 [==============================] - 345s 884ms/step - loss: 0.2382 - accuracy: 0.9070


In [ ]:
score, accuracy = model.evaluate(X_test_clamped, y_test, batch_size = batch_size)
print("Точность на тестовой выборке: "+str(accuracy*100)+"%.")

391/391 [==============================] - 94s 238ms/step - loss: 0.3071 - accuracy: 0.8750
Точность на тестовой выборке: 87.5%.
